In [1]:
from frontend.database import session
from scripts import generate_xnor_circuit, benchmark_utils
import numpy as np

In [92]:
weight_size = 200000
weight = np.asarray([int(np.random.random() > 0.5) for i in range(weight_size)])

In [93]:
circuit_file=generate_xnor_circuit.generate(weight.size, weight)

In [94]:
inp_var = benchmark_utils.get_inputs(circuit_file)
inp_dict = {}
for inps in inp_var:
    inp_dict[inps] = int(np.random.random() > 0.5)

In [ ]:
inputs_file = benchmark_utils.write_inputs_file(inp_dict)

In [ ]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"bool","TFHE")

In [ ]:
assert np.alltrue([int(weight[i]==inp_dict['INPUT_'+str(i)]) == int(results['Outputs']['OUTPUT_'+str(i)]) for i in range(weight_size)])

In [ ]:
print(results['Processing times (s)']['circuit_evaluation'])